<a href="https://colab.research.google.com/github/halfnet/BacktestR/blob/master/BFA_lost_sales_prototype_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import pandas_gbq
import datetime
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [3]:
def bleed_down_hypothetical(df):
  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row['manufacturer_part_id'] == manufacturer_part_id:
      df.at[index, 'start_qty'] = next_start_qty
      df.at[index, 'start_qty2'] = next_start_qty2
    else:
      manufacturer_part_id = row['manufacturer_part_id']
      next_start_qty = row['start_qty']
      next_start_qty2 = row['start_qty2']

    if (row['pickable_qty'] == 0) & (row['order_qty'] == 0):
      # oos and no order at all, use lost_sales to bleed down
      next_start_qty = next_start_qty + row['other_receipt_qty'] + row['hypothetical_qty'] - row['potential_lost_sales']
      next_start_qty2 = next_start_qty2 + row['other_receipt_qty'] + row['hypothetical_qty2'] - row['potential_lost_sales']
      # available qty can't be negative
      if next_start_qty < 0:
        df.at[index, 'order_qty'] = next_start_qty + row['potential_lost_sales']
        next_start_qty = 0
      else:
        df.at[index, 'order_qty'] = row['potential_lost_sales']

      if next_start_qty2 < 0:
        df.at[index, 'order_qty2'] = next_start_qty2 + row['potential_lost_sales']
        next_start_qty2 = 0
      else:
        df.at[index, 'order_qty2'] = row['potential_lost_sales']

    else:
      next_start_qty = next_start_qty + row['other_receipt_qty'] + row['hypothetical_qty'] - row['order_qty']
      next_start_qty2 = next_start_qty2 + row['other_receipt_qty'] + row['hypothetical_qty2'] - row['order_qty2']
      if next_start_qty < 0:
        df.at[index, 'order_qty'] = next_start_qty + row['order_qty']
        next_start_qty = 0

      if next_start_qty2 < 0:
        df.at[index, 'order_qty2'] = next_start_qty2 + row['order_qty2']
        next_start_qty2 = 0

  # separate hypothetical cases into two dataframes and then stack them together
  df2 = df.drop(columns=['start_qty', 'hypothetical_qty', 'order_qty'])
  df2.rename(columns={'start_qty2': 'start_qty', 'hypothetical_qty2': 'hypothetical_qty', 'order_qty2': 'order_qty'}, inplace=True)
  df2['analysis_case'] = 'hypothetical2'
  df.drop(columns=['start_qty2', 'hypothetical_qty2', 'order_qty2'], inplace=True)
  df['analysis_case'] = 'hypothetical1'
  df = pd.concat([df, df2])

  df['is_oos'] = np.where((df.start_qty + df.other_receipt_qty + df.hypothetical_qty - df.order_qty) <= 0, 1, 0)
  
  return df

In [4]:
def prototype_run(proposal_id):

  print("-------" + str(proposal_id) + "-------")
  client = bigquery.Client(project=project_id)  

  client = bigquery.Client(project=project_id)  

  sql ="""\
    SELECT 
      MAX(snapshot_id) snapshot_id,
      MAX(DATE(refresh.brstarttime)) refresh_date, 
      MAX(order_cadence) order_cadence,
      MAX(arrival_date) arrival_date
    FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` proposal
    INNER JOIN `wf-gcp-us-ae-ops-prod.buyfair_bulk.tbl_buyfair_refresh_vertica` refresh
      ON proposal.snapshot_id = refresh.brid
    WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()
  snapshot_id = df.loc[0, 'snapshot_id']
  start_date = df.loc[0, 'refresh_date']
  order_cadence = df.loc[0, 'order_cadence']
  # for now for end_date, we use later of (arrival_date + order_cadence) and today
  # we should revisit this, for example, the discontinued items should be stop at some point
  end_date = df.loc[0, 'arrival_date']
  end_date = end_date + datetime.timedelta(int(order_cadence))
  end_date = datetime.date.today() if end_date < datetime.date.today() else end_date

  ##### Delete Existing Data #####
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      WHERE proposal_id = {}
  """
  query=sql.format(proposal_id)
  df = client.query(query).to_dataframe()

  ##### Delete Data From Previous Proposals #####
  # for hypothetical case, we need to handle deletion differently
  # this is because the start date of the simulation is not same across all products
  sql ="""\
      DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype` 
      WHERE date >= '{}'
      AND analysis_case IN ('actual', 'model')
      AND manufacturer_part_id IN
        (SELECT DISTINCT manufacturer_part_id 
        FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {})
  """
  query=sql.format(start_date, proposal_id)
  df = client.query(query).to_dataframe()

  ##### Actual Case #####
  print("working on actual case...")
  sql ="""\
      WITH item_start_end_dates AS
      (
        SELECT supplier_id, manufacturer_part_id, sprid,
          CAST('{}' AS DATE) start_date,
          CAST('{}' AS DATE) end_date 
        FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` 
        WHERE proposal_id = {}
        GROUP BY 1, 2, 3
      )
      SELECT 
        'actual' analysis_case,
        {} proposal_id,
        dates.supplier_id,
        ls.manufacturer_part_id,
        dates.sprid,
        ls.date date,
        inv.pickable_qty,
        0 receipt_qty,
        ls.order_qty demand_qty,
        ls.lost_sales potential_lost_sales,
        ls.wholesale_cost_per_unit,
        ls.revenue_per_unit,
      FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg` ls
      INNER JOIN item_start_end_dates dates
        ON ls.manufacturer_part_id = dates.manufacturer_part_id 
        AND ls.date BETWEEN dates.start_date AND dates.end_date
      LEFT JOIN `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_product_inventory_daily` inv
        ON ls.date = inv.effect_date AND ls.manufacturer_part_id = inv.manufacturer_part_id;
  """
  query=sql.format(start_date, end_date, proposal_id, proposal_id)
  df = client.query(query).to_dataframe()

  df.receipt_qty = df.receipt_qty.astype(float)
  df.demand_qty = df.demand_qty.astype(float)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.date = pd.to_datetime(df.date)

  df = df.sort_values(by=['manufacturer_part_id','date'])
  df.pickable_qty = df.pickable_qty.interpolate(limit_direction='both')
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')
  df['is_oos'] = np.where(df.pickable_qty <= 0, 1, 0)

  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now
  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  sql ="""\
      INSERT INTO `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
      ) 
      SELECT
          bd.analysis_case,
          bd.proposal_id,
          bd.supplier_id,
          bd.manufacturer_part_id,
          bd.sprid,
          EXTRACT(DATE FROM bd.date),
          bd.is_oos,
          bd.pickable_qty,
          bd.receipt_qty,
          bd.demand_qty,
          bd.potential_lost_sales,
          bd.wholesale_cost_per_unit,
          bd.revenue_per_unit,
          bd.is_oos * bd.potential_lost_sales,
          bd.is_oos * bd.potential_lost_sales * bd.wholesale_cost_per_unit,
          bd.is_oos * bd.potential_lost_sales * bd.revenue_per_unit
      FROM 
          {} bd
      ;
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()  

  ##### Hypothetical Case #####
  print("working on hypothetical case...")
  sql ="""\
      WITH onhand as (
        SELECT
          supplier_id,
          proposal_id,
          sprid,
          manufacturer_part_id,
          MIN(on_hand_qty) AS on_hand,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2,3,4),
      proposal0 as (
        SELECT
          l1.manufacturer_part_id,
          l1.arrival_date AS date,
          SUM(l1.quantity) AS quantity,
          AVG(l2.quantity) AS quantity2,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal` l1
        LEFT OUTER JOIN `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l2` l2
          ON l1.proposal_id = l2.proposal_id AND l1.manufacturer_part_id = l2.manufacturer_part_id
        WHERE l1.proposal_id = {}
        GROUP BY
          1,2
      ),
      proposal as (
        SELECT
          manufacturer_part_id,
          date,
          quantity,
          quantity2,
          SUM(quantity) OVER (
            PARTITION BY manufacturer_part_id
            ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS total_qty,
        FROM proposal0
      ),
      proposal_l1 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_l2 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l2`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_l4 as (
        SELECT
          manufacturer_part_id,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_l4`
        WHERE proposal_id = {}
        GROUP BY
          1
      ),
      proposal_receipt AS (
        SELECT manufacturer_part_id, 
        DATE_ADD(receipt_date, INTERVAL 2 DAY) AS date, 
        SUM(receipt_qty) receipt_qty
        FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_spo_receipt`
        WHERE spo IN 
          (SELECT DISTINCT spo FROM
            `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_spo`
            WHERE proposal_id = {})
        GROUP BY 1, 2
      ),
      other_receipt AS (
        SELECT manufacturer_part_id, 
        DATE_ADD(receipt_date, INTERVAL 2 DAY) AS date, 
        SUM(receipt_qty) receipt_qty
        FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_spo_receipt`
        WHERE spo NOT IN 
          (SELECT DISTINCT spo FROM
            `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_spo`
            WHERE proposal_id = {})
        GROUP BY 1, 2
      ),
      date_range as (
        SELECT * FROM UNNEST(GENERATE_DATE_ARRAY('{}', '{}', INTERVAL 1 DAY)) AS date
      ),
      existing_product AS (
        SELECT DISTINCT manufacturer_part_id FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
        WHERE analysis_case LIKE 'hypothetical%' AND date < '{}'
      )
      SELECT
        CASE WHEN existing_product.manufacturer_part_id IS NULL THEN 0 ELSE 1 END existing_product,
        onhand.proposal_id,
        onhand.supplier_id,
        onhand.manufacturer_part_id,
        onhand.sprid,
        date_range.date,
        CASE WHEN date_range.date = '{}' THEN on_hand ELSE 0 END as start_qty,
        IFNULL(other_receipt.receipt_qty, 0) AS other_receipt_qty,
        IFNULL(proposal_receipt.receipt_qty, 0) AS proposal_receipt_qty,
        IFNULL(proposal.quantity, 0) AS proposal_qty,
        IFNULL(1.0 * proposal.quantity2 * proposal.quantity / proposal.total_qty, 0) AS proposal_qty2,
        IFNULL(proposal_l1.quantity, 0) AS proposal_l1_qty,
        IFNULL(proposal_l2.quantity, 0) AS proposal_l2_qty,
        IFNULL(proposal_l4.quantity, 0) AS proposal_l4_qty,
        SUM(IFNULL(proposal_receipt.receipt_qty, 0))
        OVER (
          PARTITION BY onhand.manufacturer_part_id
          ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
        ) AS proposal_receipt_total_qty,
        ls.order_qty order_qty,
        inv.pickable_qty,
        inv.on_hand_qty,
        ls.lost_sales potential_lost_sales,
        ls.wholesale_cost_per_unit,
        ls.revenue_per_unit,   
      FROM
        onhand
      CROSS JOIN
        date_range
      LEFT JOIN 
        proposal 
      ON
        proposal.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal.date = date_range.date
      LEFT JOIN 
        proposal_l1
      ON
        proposal_l1.manufacturer_part_id = onhand.manufacturer_part_id
      LEFT JOIN 
        proposal_l2
      ON
        proposal_l2.manufacturer_part_id = onhand.manufacturer_part_id
      LEFT JOIN 
        proposal_l4
      ON
        proposal_l4.manufacturer_part_id = onhand.manufacturer_part_id 
      LEFT JOIN 
        proposal_receipt 
      ON
        proposal_receipt.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal_receipt.date = date_range.date
      LEFT JOIN 
        other_receipt 
      ON
        other_receipt.manufacturer_part_id = onhand.manufacturer_part_id 
        AND other_receipt.date = date_range.date
      LEFT JOIN 
        `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg` ls
      ON 
        ls.manufacturer_part_id = onhand.manufacturer_part_id
        AND date_range.date = ls.date
      LEFT JOIN 
        `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_product_inventory_daily` inv
        ON inv.manufacturer_part_id = onhand.manufacturer_part_id
        AND date_range.date = inv.effect_date
      LEFT JOIN 
        existing_product
        ON existing_product.manufacturer_part_id = onhand.manufacturer_part_id
      ORDER BY
        onhand.manufacturer_part_id ASC,
        date_range.date ASC;
  """
  query=sql.format(proposal_id, proposal_id, proposal_id, proposal_id,
                  proposal_id, proposal_id, proposal_id,   
                  start_date, end_date, start_date, start_date)
  df = client.query(query).to_dataframe()

  # the following columns are duplicated to support hypothetical case 1 and 2
  #     start_qty
  #     order_qty
  #     hypothetical_qty
  # fix data types
  df.start_qty = df.start_qty.astype(float)
  df['start_qty2'] = df.start_qty
  df.other_receipt_qty = df.other_receipt_qty.astype(float)
  df.order_qty = df.order_qty.astype(float)
  df['order_qty2'] = df.order_qty
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.date = pd.to_datetime(df.date)
  df.pickable_qty = df.pickable_qty.astype(float)
  df.on_hand_qty = df.on_hand_qty.astype(float)

  # fill missing data
  df.pickable_qty = df.pickable_qty.interpolate(limit_direction='both')
  df.on_hand_qty = df.on_hand_qty.interpolate(limit_direction='both')
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')

  # if products proposed but are not ordered or not arrived, simply use proposed qty and date it suppose to arrive
  # once arrived, we need to distribute the proposed qty based on actual receipt qtys and dates
  df['hypothetical_qty'] = np.where(df.proposal_receipt_total_qty == 0, df.proposal_qty, 
                                    1.0 * df.proposal_l1_qty * df.proposal_receipt_qty / df.proposal_receipt_total_qty)
  df['hypothetical_qty2'] = np.where(df.proposal_receipt_total_qty == 0, df.proposal_qty2, 
                                    1.0 * df.proposal_l2_qty * df.proposal_receipt_qty / df.proposal_receipt_total_qty)

  df.drop(columns=['proposal_receipt_qty', 'proposal_qty', 'proposal_qty2', 'proposal_l1_qty', 'proposal_l2_qty',
                  'proposal_l4_qty', 'proposal_receipt_total_qty'], inplace=True)

  # now let's separate the products into 2 groups (dataframes)
  # 1 - products that don't exist before --> bleed down from refresh date
  # 2 - products that do exist --> bleed down from impact date (this can be receipt date or expected arrival date)
  #                                 then delete duplicated records
  df_new = df[df.existing_product == 0].copy()
  df_new.drop(columns=['on_hand_qty'], inplace=True)
  df_new = bleed_down_hypothetical(df_new)

  df_exist = df[df.existing_product == 1].copy()
  df_exist_product = df_exist[df_exist.hypothetical_qty>0].groupby(['manufacturer_part_id'], as_index=False).agg(
      {'date': 'min'})
  df_exist_product.rename(columns={'date': 'impact_date'}, inplace=True)
  df_exist = pd.merge(df_exist,
                      df_exist_product,
                      left_on='manufacturer_part_id',
                      right_on='manufacturer_part_id',
                      how='inner')
  df_exist = df_exist[df_exist.date >= df_exist.impact_date].copy()
  # now initialize the start qty with on_hand_qty (note: not pickable_qty)
  df_exist.start_qty = np.where(df_exist.date == df_exist.impact_date, df_exist.on_hand_qty, 0)
  df_exist.start_qty2 = df_exist.start_qty
  df_exist.drop(columns=['on_hand_qty', 'impact_date'], inplace=True)
  df_exist = bleed_down_hypothetical(df_exist)

  df = pd.concat([df_new, df_exist])
  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now
  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  # now delete older records basing on what we are about to insert
  sql ="""\
        DELETE FROM `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype` pt 
        WHERE date >= '{}'
        AND EXISTS(SELECT 1 FROM {} tmp
          WHERE pt.analysis_case = tmp.analysis_case AND pt.date = CAST(tmp.date AS DATE) AND pt.manufacturer_part_id = tmp.manufacturer_part_id)
  """
  query=sql.format(start_date, fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  # now insert new records
  sql ="""\
      INSERT INTO `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
      ) 
      SELECT
          analysis_case,
          bd.proposal_id,
          bd.supplier_id,
          bd.manufacturer_part_id,
          bd.sprid,
          EXTRACT(DATE FROM bd.date),
          bd.is_oos,
          bd.start_qty,
          bd.hypothetical_qty + bd.other_receipt_qty,
          bd.order_qty,
          bd.potential_lost_sales,
          bd.wholesale_cost_per_unit,
          bd.revenue_per_unit,
          bd.is_oos * bd.potential_lost_sales,
          bd.is_oos * bd.potential_lost_sales * bd.wholesale_cost_per_unit,
          bd.is_oos * bd.potential_lost_sales * bd.revenue_per_unit
      FROM 
          {} bd;
  """
  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()

  ###### Model Case #####
  print("working on model case...")
  sql ="""\
      WITH demand_forecast AS (
        SELECT
          manufacturer_part_id,
          EXTRACT(MONTH FROM start_date) AS df_month,
          EXTRACT(YEAR FROM end_date) AS df_year,
          SUM(forecast_qty_daily) AS df_qty
        FROM
          `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_demand_forecast_cg`
        WHERE
          snapshot_id = {}
        GROUP BY
          1,2,3),
      onhand as (
        SELECT
          supplier_id,
          proposal_id,
          sprid,
          manufacturer_part_id,
          MIN(on_hand_qty) AS on_hand,
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2,3,4),
      proposal as (
        SELECT
          manufacturer_part_id,
          arrival_date AS date,
          SUM(quantity) AS quantity
        FROM
          `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal`
        WHERE proposal_id = {}
        GROUP BY
          1,2
      ),
      inbound as (
        SELECT
          manufacturer_part_id,
          estimated_pickable_date,
          SUM(quantity) inbound_qty,
        FROM
          `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_inbound_shipment`
        WHERE snapshot_id = {}
        GROUP BY
          1,2
      ),
      date_range as (
        SELECT * FROM UNNEST(GENERATE_DATE_ARRAY('{}', '{}', INTERVAL 1 DAY)) AS date
      )
      SELECT
        onhand.proposal_id,
        onhand.supplier_id,
        onhand.manufacturer_part_id,
        onhand.sprid,
        date_range.date,
        CASE WHEN date_range.date = '{}' THEN on_hand ELSE 0 END AS start_qty,
        IFNULL(inbound.inbound_qty, 0) AS inbound_qty,
        IFNULL(quantity, 0) AS proposal_qty,
        df.df_qty AS forecast_qty,
        ls.lost_sales potential_lost_sales,
        ls.wholesale_cost_per_unit,
        ls.revenue_per_unit,        
      FROM
        onhand
      CROSS JOIN
        date_range
      LEFT JOIN
        demand_forecast AS df
      ON
        onhand.manufacturer_part_id = df.manufacturer_part_id
        AND EXTRACT(MONTH FROM date_range.date) = df.df_month
        AND EXTRACT(YEAR FROM date_range.date) = df.df_year
      LEFT JOIN 
        proposal 
      ON
        proposal.manufacturer_part_id = onhand.manufacturer_part_id 
        AND proposal.date = date_range.date
      LEFT JOIN 
        inbound 
      ON
        inbound.manufacturer_part_id = onhand.manufacturer_part_id 
        AND inbound.estimated_pickable_date = date_range.date
      LEFT JOIN `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.vw_lost_sales_cg` ls
        ON 
            ls.date = date_range.date
            AND ls.manufacturer_part_id = onhand.manufacturer_part_id
      ORDER BY
        onhand.manufacturer_part_id ASC,
        date_range.date ASC;
  """

  query=sql.format(snapshot_id, proposal_id, proposal_id, snapshot_id, start_date, end_date, start_date)
  df = client.query(query).to_dataframe()

  df.start_qty = df.start_qty.astype(float)
  df.inbound_qty = df.inbound_qty.astype(float)
  df.proposal_qty = df.proposal_qty.astype(float)
  df.forecast_qty = df.forecast_qty.astype(float)
  df.date = pd.to_datetime(df.date)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.astype(float)
  df.revenue_per_unit = df.revenue_per_unit.astype(float)
  df.potential_lost_sales = df.potential_lost_sales.astype(float)
  df.wholesale_cost_per_unit = df.wholesale_cost_per_unit.interpolate(limit_direction='both')
  df.revenue_per_unit = df.revenue_per_unit.interpolate(limit_direction='both')

  manufacturer_part_id = -1
  for index, row in df.iterrows():
    if row['manufacturer_part_id'] == manufacturer_part_id:
      df.at[index, 'start_qty'] = next_start_qty
    else:
      manufacturer_part_id = row['manufacturer_part_id']
      next_start_qty = row['start_qty']
    next_start_qty = next_start_qty + row['inbound_qty'] + row['proposal_qty'] - row['forecast_qty']
    next_start_qty = 0 if next_start_qty < 0 else next_start_qty

  df['is_oos'] = np.where((df.start_qty + df.inbound_qty + df.proposal_qty - df.forecast_qty) <= 0, 1, 0)

  now = datetime.datetime.now()
  now = now.strftime("%Y%m%d%H%M%S")
  tmp_table_name = "tmp_lost_sales_bleeddown_" + now
  df.to_gbq(destination_table=dataset+"."+tmp_table_name, project_id=project_id, if_exists="replace")
  fq_tmp_table_name = "`" + project_id + "." + dataset + "." + tmp_table_name + "`"

  sql ="""\
      INSERT INTO `wf-gcp-us-ae-buyfair-prod.analytics_lost_sales.tbl_lost_sales_prototype`
      (
          analysis_case,
          proposal_id,
          supplier_id,
          manufacturer_part_id,
          sprid,
          date,
          is_oos,
          available_qty,
          receipt_qty,
          demand_qty,
          potential_lost_sales,
          wholesale_cost_per_unit,
          revenue_per_unit,
          expected_lost_sales,
          expected_lost_sales_by_cost,
          expected_lost_sales_by_revenue
      ) 
      SELECT
          'model',
          bd.proposal_id,
          bd.supplier_id,
          bd.manufacturer_part_id,
          bd.sprid,
          EXTRACT(DATE FROM bd.date),
          bd.is_oos,
          bd.start_qty,
          bd.proposal_qty + bd.inbound_qty,
          bd.forecast_qty,
          bd.potential_lost_sales,
          bd.wholesale_cost_per_unit,
          bd.revenue_per_unit,
          bd.is_oos * bd.potential_lost_sales,
          bd.is_oos * bd.potential_lost_sales * bd.wholesale_cost_per_unit,
          bd.is_oos * bd.potential_lost_sales * bd.revenue_per_unit
      FROM 
          {} bd
      ;
  """

  query=sql.format(fq_tmp_table_name)
  df = client.query(query).to_dataframe()

  sql ="""\
    DROP TABLE IF EXISTS {}.{};
  """
  query=sql.format(dataset, tmp_table_name)
  df = client.query(query).to_dataframe()

In [5]:
begin_time = datetime.datetime.now()

In [6]:
project_id = "wf-gcp-us-ae-buyfair-prod"
dataset = "analytics_lost_sales"

In [7]:
client = bigquery.Client(project=project_id)  
sql ="""\
    SELECT *
    FROM `wf-gcp-us-ae-ops-prod.buyfair_stream.vw_buyfair_supply_proposal_used`
    ORDER BY snapshot_id
"""
query=sql.format()
df = client.query(query).to_dataframe()
proposals = df.proposal_id.tolist()

In [8]:
# change the proposal id and run the script by Ctrl+F10
#proposals = [5799107, 5952237]
#proposals = [5801358, 5904781]
#proposals = [5756686, 5850428]
#proposals = [5996649]

In [9]:
for proposal_id in proposals:
  prototype_run(proposal_id)

-------5748095-------
working on actual case...


1it [00:06,  6.94s/it]


working on hypothetical case...


1it [00:07,  7.26s/it]


working on model case...


...

1it [00:03,  3.12s/it]


working on hypothetical case...


1it [00:05,  5.84s/it]


working on model case...


1it [00:09,  9.16s/it]


In [10]:
print(datetime.datetime.now() - begin_time)

3:03:41.947421
